## Find Asset and Create Attachment

In [1]:
# Find existing Asset and create an attachment
#
# Main function, establishes a connection to RKVST using an App Registration then uses that
# to find existing Asset and create an attachment.
#
# The attachment is an image located in a folder named "test_files"
#
# Note: The purpose of RKVST Jupyter Notebooks is to provide simplified examples that one can easily execute and digest.
# The RKVST Python SDK is authored to work cleanly with more advanced coding techniques.
#
# RKVST Python SDK: https://github.com/rkvst/rkvst-python
#

In [2]:
from json import dumps as json_dumps
from os import getenv
from warnings import filterwarnings

from archivist.archivist import Archivist
from archivist.proof_mechanism import ProofMechanism
from archivist.logger import set_logger

filterwarnings("ignore", message="Unverified HTTPS request")

try:
    import importlib.resources as pkg_resources
except ImportError:
    import importlib_resources as pkg_resources

import test_files

In [3]:
# URL, CLIENT, SECRET are environment variables that represent connection parameters.  
#
# URL = represents the url to the RKVST application
# CLIENT = represents the client ID from an Application Registration
# SECRET = represents the client secret from an Application Registration
RKVST_URL = getenv("URL")
APPREG_CLIENT = getenv("CLIENT")
APPREG_SECRET = getenv("SECRET")

In [4]:
def upload_attachment(arch, path, name):
    """
    Obtains attachments from "test_files" folder and creates json payload
    for upload
    """
    with pkg_resources.open_binary(test_files, path) as fd:
        blob = arch.attachments.upload(fd)
        attachment = {
            "arc_display_name": name,
            "arc_attachment_identity": blob["identity"],
            "arc_hash_value": blob["hash"]["value"],
            "arc_hash_alg": blob["hash"]["alg"],
        }
        return attachment

In [5]:
def get_artist(arch, name, artistid):
    """
    Finds existing Artist asset by name and unique id
    """
    attrs = {
        "arc_display_type": "Artists",
        "arc_display_name": name,
        "artistid": artistid,
    }

    return arch.assets.read_by_signature(attrs=attrs)

In [6]:
def create_image(arch, asset):
    """
    Creates an event that uploads a primary image for an Artist asset
    """
    attachments = upload_attachment(
        arch, "pexels-andrea-turner-707697.jpeg", "arc_primary_image"
    )
    attachment_list = [attachments]

    props = {"operation": "Record", "behaviour": "RecordEvidence"}
    attrs = {
        "arc_description": "Attaching an image",
        "arc_display_type": "Primary image",
    }

    asset_attrs = {"arc_attachments": attachment_list}

    return arch.events.create(
        asset["identity"], props=props, attrs=attrs, asset_attrs=asset_attrs
    )

In [7]:
"""
Main function of asset and event creation.

* Connect to RKVST with client ID and client secret
* Finds existing asset
* Creates additional event for attachment upload
* Prints response of asset and event creation
"""
# Optional call to set the logger level.  The argument can be either
# "INFO" or "DEBUG".  For more sophisticated logging control see our
# documentation.
set_logger("INFO")

# Initialize connection to RKVST
print("Connecting to RKVST")
print("URL", RKVST_URL)
arch = Archivist(RKVST_URL, (APPREG_CLIENT, APPREG_SECRET), max_time=300)

Connecting to RKVST
URL https://app.rkvst.io


In [8]:
# Finding existing artist by name and artist id
# Fill in the artistid from Create Asset and Events notebook
print("Finding Asset")
asset = get_artist(arch, "Adele Laurie Blue Adkins", "7lamw1s0xxl9")
print("Asset", json_dumps(asset, indent=4))

Finding Asset


Refresh token


Asset {
    "identity": "assets/afb30753-89ef-4c2c-a0e8-3ebabd52a5b8",
    "behaviours": [
        "Attachments",
        "RecordEvidence",
        "Builtin",
        "AssetCreator"
    ],
    "attributes": {
        "genre": "Soul",
        "stage_name": "Adele",
        "arc_description": "British Soul Singer",
        "arc_display_name": "Adele Laurie Blue Adkins",
        "arc_display_type": "Artists",
        "artistid": "7lamw1s0xxl9"
    },
    "confirmation_status": "CONFIRMED",
    "tracked": "TRACKED",
    "owner": "0x5284e740A744F075E402f7fB0c4485532ddf4Af8",
    "at_time": "2022-12-21T21:29:37Z",
    "storage_integrity": "TENANT_STORAGE",
    "proof_mechanism": "SIMPLE_HASH",
    "chain_id": "8275868384",
    "public": false,
    "tenant_identity": "tenant/0a62f7c9-fd7b-4791-8041-01218d839ec1"
}


In [9]:
# Create an event for attachment upload
print("Creating Events for existing Asset")
image_event = create_image(arch, asset)
print("Event for Image", json_dumps(image_event, indent=4))

Creating Events for existing Asset
Event for Image {
    "identity": "assets/afb30753-89ef-4c2c-a0e8-3ebabd52a5b8/events/11f775e5-1fde-42d7-a8c5-af9477c50ffa",
    "asset_identity": "assets/afb30753-89ef-4c2c-a0e8-3ebabd52a5b8",
    "event_attributes": {
        "arc_description": "Attaching an image",
        "arc_display_type": "Primary image"
    },
    "asset_attributes": {
        "arc_attachments": [
            {
                "arc_hash_alg": "SHA256",
                "arc_hash_value": "cf0dd630dcfb6e2eac65c362bf7d5ff382a4241ebf45a69a2541ee43320d4af6",
                "arc_attachment_identity": "blobs/6d1fd3ee-bf63-49a8-8960-61d7a6ea1605",
                "arc_display_name": "arc_primary_image"
            }
        ]
    },
    "operation": "Record",
    "behaviour": "RecordEvidence",
    "timestamp_declared": "2022-12-21T21:29:45Z",
    "timestamp_accepted": "2022-12-21T21:29:45Z",
    "timestamp_committed": "2022-12-21T21:29:46.040010324Z",
    "principal_declared": {
     